In [1]:
import osmnx as ox
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import json

def column_usefulness(df,perc=50):
    print(len(df))
    for i,n in df.isna().sum().sort_values().iteritems():
        if n<perc/100*len(df):
            print(f"{i}: {(len(df)-n)/len(df):.2%}")

# Create CityScope Table

In [2]:
from brix import Grid_maker, Handler
table_name = 'ntt_rome'
lat,lon = 41.835720, 12.498408 #NTTD IT Rome Coordinates (top-left point)
G = Grid_maker(table_name, lat, lon, 
               cell_size=75, nrows=20, ncols=20, rotation=0, 
               crs_epsg=None, quietly=False)

https://cityio.media.mit.edu/api/tables/list/ Attempt: 0
Table ntt_rome already exists
Calculating initial coordinates of each cell
Defining properties and headers


/opt/anaconda3/envs/prox/lib/python3.10/site-packages/brix/grid_maker.py:82: UserWarning: Table ntt_rome already exists
  warn(f'Table {self.table_name} already exists')


In [3]:
grid_geo=G.get_grid_geojson()
grid_gdf=gpd.read_file(json.dumps(grid_geo))

Generating grid geojson


## Assign Land Uses to grid cells

Have to ignore non-polygon types

In [4]:
db_valid= gpd.read_file("zones/ntt.json")   #as is scenario (from OSM)

In [5]:
grid_over_lu=gpd.overlay(grid_gdf, db_valid)

In [6]:
grid_gdf=grid_gdf.merge(grid_over_lu[['id', 'section']], 
                        left_on='id', right_on='id', how='left')

In [7]:
grid_gdf['section'] = grid_gdf['section'].fillna(value='none')

In [8]:
grid_gdf['name']=grid_gdf['section'].copy()
grid_gdf=grid_gdf.drop(columns=['section'])

## Convert back to dict and add metadata

In [9]:
grid_gdf

,height,id,interactive,name,geometry
0,0,0,Web,residential,"POLYGON ((12.49841 41.83572, 12.49841 41.83504..."
1,0,0,Web,residential,"POLYGON ((12.49841 41.83572, 12.49841 41.83504..."
2,0,0,Web,residential,"POLYGON ((12.49841 41.83572, 12.49841 41.83504..."
3,0,0,Web,residential,"POLYGON ((12.49841 41.83572, 12.49841 41.83504..."
4,0,0,Web,residential,"POLYGON ((12.49841 41.83572, 12.49841 41.83504..."
...,...,...,...,...,...
1317,0,396,Web,residential,"POLYGON ((12.51285 41.82289, 12.51285 41.82222..."
1318,0,396,Web,park,"POLYGON ((12.51285 41.82289, 12.51285 41.82222..."
1319,0,397,Web,park,"POLYGON ((12.51375 41.82289, 12.51375 41.82222..."
1320,0,398,Web,park,"POLYGON ((12.51466 41.82289, 12.51466 41.82222..."


In [10]:
new_grid_geo=grid_gdf.__geo_interface__

In [11]:
new_grid_geo

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'height': 0,
    'id': 0,
    'interactive': 'Web',
    'name': 'residential'},
   'geometry': {'type': 'Polygon',
    'coordinates': (((12.498408000000001, 41.83572),
      (12.498408000283233, 41.83504483911095),
      (12.4993108056971, 41.83504483590043),
      (12.499310805413868, 41.835719996789486),
      (12.498408000000001, 41.83572)),)},
   'bbox': (12.498408000000001,
    41.83504483590043,
    12.4993108056971,
    41.83572)},
  {'id': '1',
   'type': 'Feature',
   'properties': {'height': 0,
    'id': 0,
    'interactive': 'Web',
    'name': 'residential'},
   'geometry': {'type': 'Polygon',
    'coordinates': (((12.498408000000001, 41.83572),
      (12.498408000283233, 41.83504483911095),
      (12.4993108056971, 41.83504483590043),
      (12.499310805413868, 41.835719996789486),
      (12.498408000000001, 41.83572)),)},
   'bbox': (12.498408000000001,
    41.83504483590043,
  

In [12]:
new_grid_geo['properties']=grid_geo['properties']

## Edit the 'Types' list

In [13]:
types={
    "educational":
    {
        "color":
        [
            1,#
            152,
            189,
            255 #180
        ],
        "height":
        [
            0,
            4,
            40
        ]
    },
    "institutional":
    {
        "color":
        [
            232,#
            254,
            181,
            255 #180
        ],
        "height":
        [
            0,
            4,
            40
        ]
    },
    "industrial":
    {
        "color":
        [
            213,#
            2,
            85,
            255 #180
        ],
        "height":
        [
            0,
            4,
            40
        ]
    },
    
    "office":
    {
        "color":
        [
            227,
            26,
            28,
            255 #180
        ],
        "height":
        [
            0,
            4,
            40
        ]
    },
    "park":
    {

        "color":#
        [
            0,
            168,
            157,
            180 #180
        ],
        "height":
        [
            0,
            4,
            40
        ],
    },
    "residential":
    {
        "color":
        [
            254,#
            173,
            84,
            180 #180
        ],
        "height":
        [
            0,
            4,
            40
        ]
    },
    "retail":
    {
        "color":
        [
            251,
            154,
            153,
            180 #180
        ],
        "height":
        [
            0, # min
            4, # default
            40 # max
        ]
    }, 
    "none":
    {
        "color":
        [
            0,
            0,
            0,
            0
        ],
        "height": 0
    }
}

In [14]:
for t in types:
    types[t]['name']=t

In [15]:
new_grid_geo['properties']['types']=types

In [16]:
G.geojson_object=new_grid_geo

In [17]:
try:
    G.commit_grid()
except:
    pass

https://cityio.media.mit.edu/api/tables/list/ Attempt: 0


Table exists, do you want to overwrite it? yes


Geogrid posted to:
https://cityio.media.mit.edu/api/table/ntt_rome/
200
https://cityio.media.mit.edu/api/table/ntt_rome/GEOGRID/ Attempt: 0
Proceeding without reviewing GEOGRIDDATA
https://cityio.media.mit.edu/api/table/ntt_rome/meta/hashes/ Attempt: 0
GEOGRIDDATA successfully updated: DbUKV162H2Q8FaQ2oAth1QLZo2epcE9kfz3kDMMaNp2p
https://cityio.media.mit.edu/api/table/ntt_rome/GEOGRIDDATA/ Attempt: 0


/opt/anaconda3/envs/prox/lib/python3.10/site-packages/brix/classes.py:981: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  geos = gpd.GeoDataFrame(geos.drop('geometry',1),geometry=geos['geometry'].apply(lambda x: shape(x))) # no crs to avoid warning
/opt/anaconda3/envs/prox/lib/python3.10/site-packages/brix/classes.py:251: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  geogrid_data = gpd.GeoDataFrame(geogrid_data.drop('geometry',1),geometry=geogrid_data['geometry'].apply(lambda x: shape(x)),crs='EPSG:4326')


In [18]:
#Table is now accessible at https://cityscope.media.mit.edu/CS_cityscopeJS/?cityscope=ntt_rome

After changing the grid on the front-end you can reset it using the following:

In [19]:
H = Handler('ntt_rome')
H.reset_geogrid_data()

https://cityio.media.mit.edu/api/tables/list/ Attempt: 0
https://cityio.media.mit.edu/api/table/ntt_rome/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/ntt_rome/GEOGRID/ Attempt: 0
Proceeding without reviewing GEOGRIDDATA
https://cityio.media.mit.edu/api/table/ntt_rome/meta/hashes/ Attempt: 0
GEOGRIDDATA successfully updated: DbUKV162H2Q8FaQ2oAth1QLZo2epcE9kfz3kDMMaNp2p
